<!--NAVIGATION-->
< [Aggregations: Min, Max, and Everything In Between](02.04-Computation-on-arrays-aggregates.ipynb) | [Contents](Index.ipynb) | [Comparisons, Masks, and Boolean Logic](02.06-Boolean-Arrays-and-Masks.ipynb) >

# 数组的计算： 广播（Broadcasting）

我们之前介绍了NumPy如何通过通用函数的**向量化操作**来减少缓慢的Python循环，另外一种向量化操作的方法是利用**NumPy的广播功能**。

广播可以简单理解为用于**不同大小**数组的二进制通用函数（加、减、乘等）的一组规则。

## 广播的介绍

前面曾提到，对于同样大小的数组，二进制操作是对相应元素逐个计算：

In [29]:
import numpy as np

In [42]:
a = np.array([0, 1, 2])
b = np.array([5, 5, 5])
a + b

array([5, 6, 7])

广播允许这些二进制操作可以用于不同大小的数组。

例如，可以简单地将一个标量（可以认为是一个零维的数组）和一个数组相加：

In [43]:
a + 5

array([5, 6, 7])

我们可以认为这个操作是将数值``5``扩展或重复至数组``[5, 5, 5]``，然后执行加法。 这种对值的重复实际上并没有发生，但是这是一种很好的理解广播的方式。

我们同样也可以将这个原理扩展到更高维度的数组。观察以下将一个一维数组和一个二维数组相加的结果：

In [44]:
M = np.ones((3, 3))
M

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [46]:
M + a

array([[1., 2., 3.],
       [1., 2., 3.],
       [1., 2., 3.]])

这里这个一维数组``a``就被扩展或者广播了。它沿着第二个维度扩展，扩展到匹配``M``数组的形状。

以上的这些例子理解起来都相对容易，更复杂的情况会涉及对两个数组的同时广播，举例：

In [34]:
a = np.arange(3)
b = np.arange(3)[:, np.newaxis] #实现shape的转化

print(a)
print(b)

[0 1 2]
[[0]
 [1]
 [2]]


In [35]:
a + b

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

正如此前将一个值扩展或广播以匹配另外一个数组的形状，这里将``a``和``b``都进行了扩展来匹配一个公共的形状，最终的结果是一个二维数组。

以上这些例子的几何可视化如下图所示：

![Broadcasting Visual](figures/02.05-broadcasting.png)

浅色的盒子表示广播的值。同样需要注意的是，这个额外的内存并没有在实际操作中进行分配，但是这样的想象方式更方便我们从概念上理解。

## 广播的规则

NumPy 的广播遵循一组严格的规则，设定这组规则是为了**决定两个数组间的操作**。

• 规则 1：如果两个数组的**维度数**不相同，那么小维度数组的形状将会在最左边补 1。（**注意**：维度数的不同指：比如一维数组、二维数组之间的不同，而不是指比如ones((2,3))和ones(3,4)的不同，因两者都是二维数组）

• 规则 2：如果两个数组的形状在任何一个维度上都不匹配，那么数组的形状会沿着维度为 1 的维度扩展以匹配另外一个数组的形状。

• 规则 3：如果两个数组的形状在任何一个维度上都不匹配并且没有任何一个维度等于 1，那么会引发异常。

为了更清楚地理解这些规则，来看几个具体示例。

### 1. 广播示例1

将一个二维数组与一个一维数组相加：

In [36]:
M = np.ones((2, 3))
a = np.arange(3)

来看这两个数组的加法操作。两个数组的形状如下：

- ``M.shape = (2, 3)`` #对于二维数组，shape输出(n,m)，其中，第一个元素n代表中一维数组中元素的个数;m代表第二维度中元素的个数。
- ``a.shape = (3,)`` #对于一维数组：这里的3表示的是第一个维度中元素的大小（size）

可以看到，根据规则1，数组``a``的维度数更小，所以在其左边补1：

- ``M.shape -> (2, 3)``
- ``a.shape -> (1, 3)``

根据规则 2，第一个维度不匹配，因此扩展这个维度以匹配数组：

- ``M.shape -> (2, 3)``
- ``a.shape -> (2, 3)``

现在两个数组的形状匹配了，可以看到它们的最终形状都为``(2, 3)``:

In [37]:
M + a

array([[1., 2., 3.],
       [1., 2., 3.]])

### 2. 广播示例2

来看两个数组均需要广播的示例：

In [38]:
a = np.arange(3).reshape((3, 1))
b = np.arange(3)

同样，首先写出两个数组的形状：

- ``a.shape = (3, 1)``
- ``b.shape = (3,)``

规则1告诉我们，需要用1将``b``的形状补全：

- ``a.shape -> (3, 1)``
- ``b.shape -> (1, 3)``

规则 2告诉我们，需要更新这两个数组的维度来相互匹配：

- ``a.shape -> (3, 3)``
- ``b.shape -> (3, 3)``

因为结果匹配，所以这两个形状是兼容的，可以看到以下结果：

In [39]:
a + b

array([[0, 1, 2],
       [1, 2, 3],
       [2, 3, 4]])

### 3. 广播示例3

现在来看一个两个数组不兼容的示例：

In [40]:
M = np.ones((3, 2))
a = np.arange(3)

和第一个示例相比，这里有个微小的不同之处：矩阵 M 是转置的。那么这将如何影响计算
呢？两个数组的形状如下：

- ``M.shape = (3, 2)``
- ``a.shape = (3,)``

同样，规则1告诉我们，``a``数组的形状必须用1进行补全：

- ``M.shape -> (3, 2)``
- ``a.shape -> (1, 3)``

根据规则2，``a``数组的第一个维度进行扩展以匹配``M``的维度：

- ``M.shape -> (3, 2)``
- ``a.shape -> (3, 3)``

现在需要用到规则 3——最终的形状还是不匹配，因此这两个数组是不兼容的。当我们执行运算时会看到以下结果：

In [41]:
M + a

ValueError: operands could not be broadcast together with shapes (3,2) (3,) 

请注意，这里可能发生的混淆在于：你可能想通过在``a``数组的右边补1，而不是左边补1，让``a``和``M``的维度变得兼容。

但是这不被广播的规则所允许。这种灵活性在有些情景中可能会有用，但是它可能会导致结果模糊。

如果你希望实现右边补全，可以通过变形数组来实现（将会用到``np.newaxis``关键字，详情请参见 2.2节）：

In [ ]:
a[:, np.newaxis].shape

In [ ]:
M + a[:, np.newaxis]

另外也需要注意，这里仅用到了``+``运算符，而这些广播规则对于任意二进制通用函数都是适用的。例如这里的``logaddexp(a, b)``函数，比起简单的方法，该函数计算``log(exp(a) + exp(b))``更准确：

In [ ]:
np.logaddexp(M, a[:, np.newaxis])

## 广播的实际应用

下面，我们将通过几个简单的示例来展示广播功能的作用。

### 1. 数组的归一化（Centering an array）

在前面的一节中，我们看到**通用函数**让NumPy用户免于写很慢的Python循环。

广播进一步扩展了这个功能，一个常见的例子就是**数组数据的归一化**。

假设你有一个有10个观察值的数组，每个观察值包含3个数值，我们将用一个10×3的数组存放该数据：

In [ ]:
X = np.random.random((10, 3))

我们可以计算每个特征的均值，计算方法是利用``mean``函数沿着第一个维度聚合：

In [ ]:
Xmean = X.mean(0)
Xmean

现在通过从``X``数组的元素中减去这个均值实现归一化（该操作是一个广播操作）：

In [ ]:
X_centered = X - Xmean

为了进一步核对我们的处理是否正确，可以查看归一化的数组的均值是否接近0：

In [ ]:
X_centered.mean(0)

在机器精度范围内，该均值为0。

### 2. 画一个二维函数

广播另外一个非常有用的地方在于能**基于二维函数显示图像**。

我们希望定义一个函数$z = f(x, y)$，可以用广播沿着数值区间计算该函数：

In [ ]:
# x和y表示0~5区间50个步长的序列
x = np.linspace(0, 5, 50)
y = np.linspace(0, 5, 50)[:, np.newaxis]

z = np.sin(x) ** 10 + np.cos(10 + y * x) * np.cos(x)

我们将用 Matplotlib 来画出这个二维数组：

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(z, origin='lower', extent=[0, 5, 0, 5],
           cmap='viridis')
plt.colorbar();

<!--NAVIGATION-->
< [Aggregations: Min, Max, and Everything In Between](02.04-Computation-on-arrays-aggregates.ipynb) | [Contents](Index.ipynb) | [Comparisons, Masks, and Boolean Logic](02.06-Boolean-Arrays-and-Masks.ipynb) >